In [1]:
import pandas as pd

In [2]:
file_name= 'seoul_real_estate_selected_merged_20250725_141605.csv'

In [14]:
df = pd.read_csv(file_name)

C:\Users\4-2025010176\AppData\Local\Temp\ipykernel_32140\707990406.py:1: DtypeWarning: Columns (7,15,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [ ]:
target_gdf = df.groupby(['CGG_NM', 'CGG_CD']).size().reset_index(name='CNT')

In [27]:
import folium
import geopandas as gpd
import pandas as pd
from branca.colormap import linear

# 1. 서울시 구 경계 GeoJSON 읽기
gdf_gu = gpd.read_file('./geodata/seoul_gu_boundaries.geojson')

# 2. target_gdf 예시 (SGG 이름과 CNT)
# 실제 데이터로 바꿔서 사용하세요
# target_gdf = pd.DataFrame({
#     'SGG': ['서초구', '송파구', '영등포구'],
#     'CNT': [30, 20, 40]  # 점 크기나 색상 척도로 활용
# })

target_gdf['CGG_CD'] = target_gdf['CGG_CD'].astype('string').str[:5]

# 3. 지도 생성 (서울 중심)
m = folium.Map(location=[37.56, 126.98], zoom_start=11, tiles='cartodb dark_matter')

# 4. 색상 스케일 생성 (CNT 값 기준)
colormap = linear.YlOrRd_09.scale(target_gdf['CNT'].min(), target_gdf['CNT'].max())
colormap.caption = 'CNT 값에 따른 색상'
colormap.add_to(m)

# 5. 각 구별로 GeoDataFrame에서 필터링 및 스타일링해서 Folium에 추가
for i, row in target_gdf.iterrows():
    sgg_name = row['CGG_NM']
    cnt = row['CNT']

    # GeoDataFrame에서 해당 구 필터링
    gu_geom = gdf_gu[gdf_gu['name'] == sgg_name]

    if gu_geom.empty:
        continue
    
    # GeoJson을 위한 스타일 함수
    def style_function(feature):
        return {
            'fillColor': colormap(cnt),
            'color': 'white',
            'weight': 1,
            'fillOpacity': 0.7,
        }

    folium.GeoJson(
        gu_geom,
        name=sgg_name,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['구 이름'])
    ).add_to(m)

    # 중심점 좌표
    centroid = gu_geom.geometry.centroid.iloc[0]
    
    # 점 크기 및 팝업/툴팁 추가 (크기는 cnt 비례)
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=cnt / 10000,  # 적당히 조절
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"{sgg_name}: {cnt}",
        tooltip=f"{sgg_name}: {cnt}"
    ).add_to(m)

# 6. 지도 출력
m


C:\Users\4-2025010176\AppData\Local\Temp\ipykernel_32140\3006523070.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gu_geom.geometry.centroid.iloc[0]
C:\Users\4-2025010176\AppData\Local\Temp\ipykernel_32140\3006523070.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gu_geom.geometry.centroid.iloc[0]
C:\Users\4-2025010176\AppData\Local\Temp\ipykernel_32140\3006523070.py:54: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gu_geom.geometry.centroid.iloc[0]
C:\Users\4-2025010176\AppData\Local\Temp\ipykernel_32140\3006523070.py:54: User